In [8]:
import numpy as np
import tensorflow as tf
import cv2
# from tensorflow.keras.models import load_model as tf_load
import matplotlib.pyplot as plt
import sys
sys.path.append("C:/Users/xl313/OneDrive/Documents/GitHub/poseTrackingXL/utils")
from load_matlab_data import loadmat_sbx
from slp_utils import posture_tracker
import scipy.io
from slp_utils import create_slp_project, crop_from_com

In [9]:
''' UPDATE data params as appropriate'''
# cam params
cam_ids = ['blue_cam', 'green_cam', 'red_cam', 'yellow_cam'] # check the input order
im_w = 2200
im_h = 650
# video params
start_frame = 35500 # in frames at 50fps # (XL, 010825: an exampler coconut caching/eating + drinking water snippet: 11:50 - 13:00 min in SLV123_110824_wEphys) 
nFrames = 2500 # in frames at 50fps # take 50sec  # used to be 2500 frames, start at 35500

''' UPDATE paths as needed '''
# videos
root_dir = "Z:/Sherry/poseTrackingXL/training_files/raw_acquisition_copy/"
vid_root = f"{root_dir}SLV123_110824_wEphys/"
# camera params
cam_params = loadmat_sbx("Z:/Sherry/poseTrackingXL/calibration_files/all_opt_arrays/102324_negated_camParams")['camParams_negateR'] #['camParams']

# to save
pred_date = "011125"
# save_file = f'{pred_date}_posture_2stage.npy' # python
save_file = f'{pred_date}_posture_2stage.mat' # matlab
save_path = f"{vid_root}{save_file}"

Z:/Sherry/poseTrackingXL/calibration_files/all_opt_arrays/102324_negated_camParams


In [10]:
# models
comNet = "Z:/Sherry/poseTrackingXL/training_files/SLP/models/010725_com250107_235615.single_instance.n=460" 
postureNet = "Z:/Sherry/poseTrackingXL/training_files/SLP/models/010825_postureNet250108_164045.single_instance.n=460"
# faceNet = "Z:\Selmaan\DPK-transfer\j4-v4"
# if running face model, otherwise set to None
# face_model = tf_load(faceNet, custom_objects={'tf': tf})
face_model = None

In [11]:
''' set up for this run '''
# set up GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [12]:
# define the video reader for each camera
all_readers = []
for i in range(len(cam_ids)):
    cam = cam_ids[i]
    print(cam)
    camPath = f"{vid_root}{cam}.avi"

    # define the video reader obj and settings
    api_id = cv2.CAP_FFMPEG
    reader = cv2.VideoCapture(camPath, api_id)
    if start_frame > 0:
        reader.set(cv2.CAP_PROP_FRAME_COUNT, start_frame)
    all_readers.append(reader)

blue_cam
green_cam
red_cam
yellow_cam


In [13]:
''' track posture '''
obj = posture_tracker(all_readers, cam_params,
                        com_model=comNet,
                        posture_model=postureNet,
                        face_model=face_model)
results = obj.track_video(start_frame=start_frame,
                            nFrames=nFrames)

cameraMats:  [array([[ 2.41702718e+03, -1.58864212e+02,  6.21403613e-01],
       [-7.25358959e+02, -1.94132218e+02,  7.01161060e-01],
       [-3.85120771e+02, -2.35069008e+03, -3.49615100e-01],
       [ 8.78300401e+05,  1.94139233e+05,  8.01105665e+02]]), array([[-2.30149020e+03,  1.39717560e+02, -7.21147459e-01],
       [ 1.05825604e+03,  1.78721097e+02, -6.07948470e-01],
       [-3.30093947e+02, -2.35235430e+03, -3.32182180e-01],
       [ 9.22424175e+05,  2.26582211e+05,  8.32531629e+02]]), array([[-7.72851025e+02, -1.00895733e+02,  7.05399101e-01],
       [-2.44458657e+03,  9.66003973e+01, -6.37337998e-01],
       [-3.51860470e+02, -2.40742776e+03, -3.10181213e-01],
       [ 8.78909726e+05,  2.54698363e+05,  8.31167280e+02]]), array([[ 1.06196576e+03,  1.60508099e+02, -6.11385346e-01],
       [ 2.31181050e+03, -2.02864429e+02,  7.18479141e-01],
       [-3.86794424e+02, -2.35450657e+03, -3.31655970e-01],
       [ 9.24258204e+05,  2.09566786e+05,  8.23582079e+02]])]
Reading and Predic

KeyboardInterrupt: 

In [7]:
# for matlab 
scipy.io.savemat(save_path,{"posture_preds": results['posture_preds'], "posture_reproj": results['posture_rep_err'],
                     "posture_rawpreds": results['posture_rawpred'], "com_preds": results['com_preds'], "com_reproj": results['com_rep_err'],
                     "posture_conf":results['posture_conf'], "com_conf":results['com_conf'], #  "face_preds":results['face_preds'], "startTime": startTime,
                     "camNames": cam_ids, "session": vid_root, "nFrames": nFrames,
                     "camParams": cam_params, }) # "rawPostures":sleap_raw_predicted_points_scale_back

In [ ]:
# # to save the ds_img (cropped from comNet) to put it in Lightning Pose
# LP_cropped_images = results["cropped_unseen_images"]
# np.shape(LP_cropped_images)
# from PIL import Image
# import os
# # Directory to save the temporary images
# output_dir = "Z:/Sherry/poseTrackingXL/training_files/posture_vids/new_images"
# os.makedirs(output_dir, exist_ok=True)
# 
# # Convert each .npy to an image and save it
# for i, img_array in enumerate(LP_cropped_images):
#     img = Image.fromarray(img_array.astype(np.uint8))  # Convert to uint8 format for images
#     img.save(os.path.join(output_dir, f"image_{i+1}.png")) 

In [ ]:
# sys.path.append("C:/Users/xl313/OneDrive/Documents/GitHub/poseTrackingXL/utils")
# from slp_utils import create_slp_project, crop_from_com
# slp_project_dir = 'Z:/Sherry/poseTrackingXL/training_files/' # this is saved locally (at least temporaily instead of on locker)
# slp_project_file = f'011125_combined_net.slp'
# slp_project_path = f'{slp_project_dir}{slp_project_file}'
# skeleton_file = 'C:/Users/xl313/OneDrive/Documents/GitHub/poseTrackingXL/postureNet/posture_skeleton_IL.csv'
# 
# training_vid_dir = 'Z:/Sherry/poseTrackingXL/training_files/'
# vid_file = f'011125_combined_pred_vid.npy'
# training_vid_path = f'{training_vid_dir}{vid_file}'
# np.save(training_vid_path, results['unseen_images'])
# # optional, create a sleap object to visualise inference results in sleap GUI.
# create_slp_project(vid_path=training_vid_path, 
#                    skeleton_file=skeleton_file,
#                    keypoints=results['sleap_raw_predicted_points_scale_back'],
#                    slp_labels_file=slp_project_path)